In [9]:
import os
import glob
import numpy as np
import awkward

In [10]:
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt

In [11]:
from sklearn.metrics import roc_auc_score, roc_curve, auc

In [12]:
# v14 8gev (updated for full 2e14 pn sample)
presel_eff = {1: 0.9952855229150378, 10: 0.9976172400798192, 100: 0.9979411114121182, 1000: 0.9981519444725636, 0: 0.04734728725337247}

In [13]:
sig_filelist = glob.glob('/home/zwan/LDMX/LDMX-scripts/GraphNet/plot_data/ddp_ecal_lite_1reg/v14_8gev_1*.parquet') + \
                 glob.glob('/home/zwan/LDMX/LDMX-scripts/GraphNet/plot_data/ddp_ecal_lite_1reg/v14_8gev_0*.parquet')
bkg_filelist = glob.glob('/home/zwan/LDMX/LDMX-scripts/GraphNet/plot_data/ddp_ecal_lite_1reg/*pn*.parquet')

In [14]:
sig_tables = [awkward.from_parquet(f) for f in sig_filelist]
#print(sig_tables[0]['ParticleNet_extra_label'])
bkg_tables = [awkward.from_parquet(f) for f in bkg_filelist]

In [15]:
load_branches = [
    'discValue_',
    'recoilX_',
    'recoilY_',
     
    'ParticleNet_extra_label',
    'ParticleNet_disc',
    'TargetSPRecoilE_pt', # use this for plotting: this is the recoil electron pT at TargetSP
    'maxPE'
]

In [16]:
def load_dict(sig_tables, bkg_tables):
    a = {}
    for k in load_branches:
        #print("Loading "+k)
        arrs = []
        for tab in sig_tables + bkg_tables:
            #print(awkward.type(tab))
            #print(tab)
            #print(tab['TargetSPRecoilE_pt'])
            #print(awkward.fields(tab))
            arr = tab[k] if k in awkward.fields(tab) else np.zeros_like(tab['ParticleNet_disc'])
            arrs.append(arr)
        a[k] = awkward.concatenate(arrs)
        if k.startswith('EcalVeto'):
            #a[k] = a[k].regular()
            a[k] = awkward.to_regular(a[k])
            if a[k].ndim==2 and len(a[k][0]) == 1:  #a[k].shape[1]==1:
                a[k] = a[k][:,0]
        print("   Found {} events".format(len(a[k])))
    return a

a = load_dict(sig_tables, bkg_tables)

print("Done")

   Found 26713756 events
   Found 26713756 events
   Found 26713756 events
   Found 26713756 events
   Found 26713756 events
   Found 26713756 events
   Found 26713756 events
Done


In [17]:
for k in a.keys():
    print(k, awkward.type(a[k]))

discValue_ 26713756 * var * float64
recoilX_ 26713756 * var * float64
recoilY_ 26713756 * var * float64
ParticleNet_extra_label 26713756 * int64
ParticleNet_disc 26713756 * float64
TargetSPRecoilE_pt 26713756 * var * float64
maxPE 26713756 * var * float64


In [18]:
def to_categorical(y, num_classes=None):
    """Converts a class vector (integers) to binary class matrix.
    E.g. for use with categorical_crossentropy.
    # Arguments
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
    # Returns
        A binary matrix representation of the input.
    """
    y = np.array(y, dtype='int').ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype='int')
    categorical[np.arange(n), y] = 1
    return categorical

def plotROC(y_preds, y_truth, sample_weight=None, output=None, labels=['signal'], sig_eff=1, bkg_eff=1, energy=0, **kwargs):
    from sklearn.metrics import auc, roc_curve, accuracy_score

    fpr = dict()
    tpr = dict()
    thresholds= dict()
    roc_auc = dict()
    outputs = {}

    plt.figure()

    for label, pred in zip(labels, y_preds):
        print("Label: ", label)
        fpr[label], tpr[label], thresholds[label] = roc_curve(y_truth, pred, sample_weight=sample_weight)
        roc_auc[label] = auc(fpr[label], tpr[label])
        fpr[label] *= bkg_eff
        tpr[label] *= sig_eff

        legend = '%s (auc* = %0.6f)' % (label, roc_auc[label])
        print(legend)
        eff = get_signal_effs(fpr[label], tpr[label], thresholds[label])
        outputs[label] = eff
        print(eff)
        plt.plot(fpr[label], tpr[label], label=legend)
#     plt.plot([0, 1], [1, 0], 'k--')
    plt.xlim(kwargs.get('xlim', [0, 1]))
    plt.ylim(kwargs.get('ylim', [0, 1]))
    plt.xlabel('False positive rate ($\epsilon_{B}$)')
    plt.ylabel('True positive rate ($\epsilon_{S}$)')
#     plt.title('Receiver operating characteristic example')
    plt.legend(loc='best')
    if kwargs.get('logy', False):
        plt.yscale('log')
    if kwargs.get('logx', False):
        plt.xscale('log')
    plt.grid()
    # TEMPORARY TITLE
    plt.title(str(k)+" MeV", fontdict = {'fontsize' : 15})
    #plt.title("All events, "+str(k)+" MeV", fontdict = {'fontsize' : 15})
    if output:
        plt.savefig(output)
#     return {'fpr':fpr, 'tpr':tpr, 'thresholds':thresholds}
    return outputs


def plotROC_multi(y_preds_, y_truth_, sample_weight_=None, output=None, labels=['signal'], sig_eff=1, bkg_eff=1, energy=0, **kwargs):
    from sklearn.metrics import auc, roc_curve, accuracy_score

    # y_preds, etc are now tuples of (a, b, c) (1reg, 2reg, 3reg)
    plt.figure()
    
    for i in range(1):  # 1, 2, 3 regions
        
        fpr = dict()
        tpr = dict()
        thresholds= dict()
        roc_auc = dict()
        outputs = {}

        #plt.figure()

        y_preds = y_preds_[i]
        y_truth = y_truth_[i]
        sample_weight = sample_weight_[i]
        print("Y_TRUTH:")
        print(y_truth[:10])
        for label, pred in zip(labels, y_preds):
            #if label == 'BDT' and i < 1:  continue
            print("Label: ", label)
            fpr[label], tpr[label], thresholds[label] = roc_curve(y_truth, pred, sample_weight=sample_weight)
            roc_auc[label] = auc(fpr[label], tpr[label])
            fpr[label] *= bkg_eff
            tpr[label] *= sig_eff
            
            if label == 'BDT':
                legend = '%s\n(auc* = %0.6f)' % (label+' (old Gabriel)', roc_auc[label])
            #elif i == 0:
                #legend = '(1,1)-reg SplitNetX\n(auc* = %0.6f)' % (roc_auc[label])
                #legend = '1-reg SplitNet\n(auc* = %0.6f)' % (roc_auc[label])
            else:
                legend = '%i-reg %s, PN \n(auc* = %0.6f)' % (i+1, label, roc_auc[label])
            #else:
                #legend = '1-reg SplitNet\n(auc* = %0.6f)' % (roc_auc[label])
            print(legend)
            eff = get_signal_effs(fpr[label], tpr[label], thresholds[label])
            outputs[label] = eff
            print(eff)
            print(len(eff))
            #print(tpr[label][:10])
            #print(len(fpr[label]))
            plt.plot(fpr[label], tpr[label], label=legend)
#     plt.plot([0, 1], [1, 0], 'k--')
    plt.xlim(kwargs.get('xlim', [0, 1]))
    plt.ylim(kwargs.get('ylim', [0, 1]))
    plt.xlabel('False positive rate ($\epsilon_{B}$)', fontsize=12)
    plt.ylabel('True positive rate ($\epsilon_{S}$)', fontsize=12)
#     plt.title('Receiver operating characteristic example')
    plt.legend(loc='best')
    if kwargs.get('logy', False):
        plt.yscale('log')
    if kwargs.get('logx', False):
        plt.xscale('log')
    plt.grid()
    # TEMPORARY TITLE
    plt.title(str(k)+" MeV w/ PN Background", fontdict = {'fontsize' : 15})
    #plt.title("All events, "+str(k)+" MeV", fontdict = {'fontsize' : 15})
    if output:
        plt.savefig(output, facecolor='w', dpi=250)
#     return {'fpr':fpr, 'tpr':tpr, 'thresholds':thresholds}
    return outputs


mistags=[1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8, 1e-9]
def get_signal_effs(fpr, tpr, thresholds, mistags=mistags):
    outputs = []
    for m in mistags:
        idx = next(idx for idx, v in enumerate(fpr) if v > m)
        outputs.append((fpr[idx], tpr[idx], thresholds[idx]))
    return outputs

In [19]:
#pT = np.array(a['TargetSPRecoilE_pt']).T[0]
test_extra_labels = a['ParticleNet_extra_label']#[pT != -999]
test_labels = test_extra_labels>0

In [20]:
roc_info = {}
#y_preds_ = [[a['ParticleNet_disc'], a['discValue_']], [b['ParticleNet_disc'], b['discValue_']], [c['ParticleNet_disc'], c['discValue_']]]
y_preds_ = [[a['ParticleNet_disc'], a['discValue_']]]
#test_labels_ = [test_labels, test_labels_b, test_labels_c]
test_labels_ = [test_labels]
#print(a['ParticleNet_disc'][:10])
for k in presel_eff:
    if k > 0:
        mass = '%d MeV' % k
        print(mass)
        weights = [np.logical_or(test_extra_labels==0, test_extra_labels==k)]
                  #np.logical_or(test_extra_labels_c==0, test_extra_labels_c==k)]
        print(type(weights))
        roc_info[k] = plotROC_multi(y_preds_, test_labels_, 
                sample_weight_=weights,
                sig_eff=presel_eff[k], bkg_eff=presel_eff[0],
                labels=['SplitNet', 'BDT'], xlim=[1e-7, 0.1], ylim=[0, 1], logx=True, energy=k)
# NOTE:  add 'BDT' to labels list if desired

1 MeV
<class 'list'>


<IPython.core.display.Javascript object>

Y_TRUTH:
[True, True, True, True, True, True, True, True, True, True]
Label:  SplitNet
1-reg SplitNet, PN 
(auc* = 0.996637)
[(0.001000039413011162, 0.9844579930153763, 0.5765608847422008), (0.00010001840876622221, 0.5605847965018925, 0.8535402055564706), (1.0079000690520947e-05, 0.15365194985012254, 0.9318076779319556), (1.012722557420765e-06, 0.022757612993321585, 0.9910168442939827), (1.446746510601093e-07, 0.013146756985535017, 0.999999829510509), (9.644976737340619e-08, 0.012039207724117863, 0.999999999961231), (9.644976737340619e-08, 0.012039207724117863, 0.999999999961231)]
7
Label:  BDT
BDT (old Gabriel)
(auc* = 0.979799)
[(0.001000039413011162, 0.758707268346348, 0.06322157382965088), (0.00010001840876622221, 0.39255153511201996, 0.9232723712921143), (1.0030775806834243e-05, 0.1416861054317823, 0.9989332556724548), (1.012722557420765e-06, 0.05056572424496209, 0.9999102354049683), (1.446746510601093e-07, 0.016768117011232323, 0.9999752044677734), (4.822488368670309e-08, 0.00092

<IPython.core.display.Javascript object>

Y_TRUTH:
[True, True, True, True, True, True, True, True, True, True]
Label:  SplitNet
1-reg SplitNet, PN 
(auc* = 0.998120)
[(0.001000039413011162, 0.9913858054955943, 0.5765608847422008), (0.00010001840876622221, 0.7717596295707976, 0.8535402055564706), (1.0079000690520947e-05, 0.42333684267922783, 0.9318076779319556), (1.012722557420765e-06, 0.14437628779209208, 0.9910168442939827), (1.446746510601093e-07, 0.0913019654888399, 0.999999829510509), (9.644976737340619e-08, 0.08544806970160322, 0.999999999961231), (9.644976737340619e-08, 0.08544806970160322, 0.999999999961231)]
7
Label:  BDT
BDT (old Gabriel)
(auc* = 0.988820)
[(0.001000039413011162, 0.8726020140890902, 0.06322157382965088), (0.00010001840876622221, 0.625598414672078, 0.9232723712921143), (1.0030775806834243e-05, 0.34066565942160965, 0.9989332556724548), (1.012722557420765e-06, 0.18261565239881372, 0.9999102354049683), (1.446746510601093e-07, 0.10365061894903119, 0.9999752044677734), (4.822488368670309e-08, 0.00476925544

<IPython.core.display.Javascript object>

Y_TRUTH:
[True, True, True, True, True, True, True, True, True, True]
Label:  SplitNet
1-reg SplitNet, PN 
(auc* = 0.998460)
[(0.001000039413011162, 0.9920230082361965, 0.5765608847422008), (0.00010001840876622221, 0.8224611314689458, 0.8535402055564706), (1.0079000690520947e-05, 0.5347240854124813, 0.9318076779319556), (1.012722557420765e-06, 0.23000642483902367, 0.9910168442939827), (1.446746510601093e-07, 0.15003688892346567, 0.999999829510509), (9.644976737340619e-08, 0.14242496313403313, 0.999999999961231), (9.644976737340619e-08, 0.14242496313403313, 0.999999999961231)]
7
Label:  BDT
BDT (old Gabriel)
(auc* = 0.990656)
[(0.001000039413011162, 0.8983177098452255, 0.06322157382965088), (0.00010001840876622221, 0.6875862242134119, 0.9232723712921143), (1.0030775806834243e-05, 0.4101831659447103, 0.9989332556724548), (1.012722557420765e-06, 0.25024516452727835, 0.9999102354049683), (1.446746510601093e-07, 0.16769807246072568, 0.9999752044677734), (4.822488368670309e-08, 0.00747427058

<IPython.core.display.Javascript object>

Y_TRUTH:
[True, True, True, True, True, True, True, True, True, True]
Label:  SplitNet
1-reg SplitNet, PN 
(auc* = 0.998615)
[(0.001000039413011162, 0.9923728339310398, 0.5765608847422008), (0.00010001840876622221, 0.8451177422403167, 0.8535402055564706), (1.0079000690520947e-05, 0.550127663394838, 0.9318076779319556), (1.012722557420765e-06, 0.23157384552730567, 0.9910168442939827), (1.446746510601093e-07, 0.14453526894784446, 0.999999829510509), (9.644976737340619e-08, 0.13399822673346007, 0.999999999961231), (9.644976737340619e-08, 0.13399822673346007, 0.999999999961231)]
7
Label:  BDT
BDT (old Gabriel)
(auc* = 0.993290)
[(0.001000039413011162, 0.9358795926489513, 0.06322157382965088), (0.00010001840876622221, 0.7934459449753593, 0.9232723712921143), (1.0030775806834243e-05, 0.5443733863996897, 0.9989332556724548), (1.012722557420765e-06, 0.302918206839736, 0.9999102354049683), (1.446746510601093e-07, 0.16779866124382056, 0.9999752044677734), (4.822488368670309e-08, 0.00859541051324

In [21]:
disc_threshold = 0.7335808

bkg_pt = np.array(a['TargetSPRecoilE_pt'][test_extra_labels==0])
bkg_disc_value = a['ParticleNet_disc'][test_extra_labels==0]
bkg_maxPE = a['maxPE'][test_extra_labels==0]

nPass = np.sum( (bkg_disc_value > disc_threshold) * (bkg_pt.T[0] != -999) )
nPass_veto = np.sum( (bkg_disc_value > disc_threshold) * (bkg_pt.T[0] != -999) * (bkg_maxPE < 8) )

print(nPass)
print(nPass_veto)

10287
0


In [22]:
masses = [0, 1, 10, 100, 1000]
pT = {}
nEvents = {}
for m in masses:
    pT[m] = np.array(a['TargetSPRecoilE_pt'][test_extra_labels==m]).T[0]
    print(len(pT[m]))
    pT[m] = pT[m][pT[m] != -999]
    nEvents[m] = len(pT[m])
    print(len(pT[m]))
print(nEvents)

981802
981802
3785062
3783143
3659756
3630028
4661474
4486595
13625662
13475484
{0: 981802, 1: 3783143, 10: 3630028, 100: 4486595, 1000: 13475484}


In [23]:
# threshold = [474896 ->10, 74805 -> 1, 1k -> 0, 100 -> 0]   # 10k -> 0
#thresholds = [0.222477, 0.7335808, 0.94611, 0.98472]  # 0.94611
#thresholds = [0.98472, 0.94611, 0.7335808, 0.222477]
thresholds = [0.94611, 0.87439, 0.7335808, 0.222477]
pT_pass = {}
for m in masses:
    disc_value = np.array(a['ParticleNet_disc'][test_extra_labels==m])
    print(disc_value.shape)
    pt = np.array(a['TargetSPRecoilE_pt'][test_extra_labels==m])
    print(pt.T[0])
    print(pt.shape)
    disc_value = disc_value[pt.T[0]!=-999]
    pT_pass[m] = [pT[m][disc_value > threshold] for threshold in thresholds]
    print(len(pT_pass[m]))

(981802,)
[1.85805321 3.27920294 0.68218046 ... 3.50796485 2.82905293 4.02590084]
(981802, 1)
4
(3785062,)
[2.18705893 6.13530159 5.38155508 ... 4.40631628 9.31886864 6.45992804]
(3785062, 1)
4
(3659756,)
[ 0.83014333  2.53539848 20.0387001  ...  2.46014738  7.35187483
 10.69134426]
(3659756, 1)
4
(4661474,)
[ 9.81662655 18.01444626 51.88372803 ... 14.30805874  0.32359737
  6.96215105]
(4661474, 1)
4
(13625662,)
[ 17.88724327  29.75032616  39.35583878 ... 605.12432861 206.91059875
 236.27598572]
(13625662, 1)
4


In [24]:
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

#colors = ['#4A7DFF', '#14AD0C', '#FF212E', '#FF8F13', '#871EFE']
colors = ['#FF212E', '#FF8F13', '#4A7DFF', '#14AD0C', '#871EFE']

for m in masses:
    fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, gridspec_kw={'height_ratios': [2, 1]})
    #if m == 1000:
        #ax1.set_ylim(100, 1300)
    vals = [pT[m], pT_pass[m]]
    bins = np.linspace(-50, 200, 51)
    #labels = ['N bkg ~ $5 \\times 10^{5} \\to 10$', 'N bkg ~ $7.5 \\times 10^{4} \\to 1$', 
               #'N bkg ~ $1 \\times 10^{3} \\to 0$', 'N bkg ~ $1 \\times 10^{2} \\to 0$']
    labels = ['N bkg ~ $1 \\times 10^{3} \\to 0$', 'N bkg ~ $1 \\times 10^{4} \\to 0$', 
             'N bkg ~ $7.5 \\times 10^{4} \\to 1$', 'N bkg ~ $5 \\times 10^{5} \\to 10$']
    n1, bins, _ = ax1.hist(vals[0], bins=bins, range=(0, 250), density=False, stacked=False, histtype='step', color=colors[-1],
                log=True, label='Inclusive')
    n2, bins, _ = ax1.hist(vals[1], bins=bins, range=(0, 250), density=False, stacked=False, histtype='step', color=colors[:-1],
                    log=True, label=labels)
    #n1, bins, _ = ax1.hist(vals[0], bins=bins, range=(0, 250), density=False, stacked=False, histtype='step', color=colors[0], 
                 #log=True, label='Inclusive')
    #n2, bins, _ = ax1.hist(vals[1], bins=bins, range=(0, 250), density=False, stacked=False, histtype='step', color=colors[3], 
                 #log=True, label=f'ParticleNetX_disc > {disc_threshold}')
    ax1.legend()
    
    ratio_arrs = []
    errs = []
    for val in n2:
        ratio_arr = val / n1
        err = (val / n1) * ( (1/np.sqrt(val)) + (1/np.sqrt(n1)) ) 
        #print(type(ratio_arr))
        #print(len(ratio_arr))       
        #print(len(bins[:-1]))
        ratio_arrs.append(ratio_arr)
        errs.append(err)
    
    for i, ratio_arr in enumerate(ratio_arrs):

        ax2.errorbar(bins[:-1],     
            ratio_arr,
            yerr=errs[i],
            fmt='o',
            alpha=0.8,
            color=colors[i])

    '''
    ax2.errorbar(bins[:-1],     # this is what makes it comparable
        n2 / n1, # maybe check for div-by-zero!
        alpha=0.8,
        yerr = (n2 / n1) * ( (1/np.sqrt(n2)) + (1/np.sqrt(n1)) ),
        fmt='o',
        color=colors[1]        
                            )
    '''
    
    ax1.set_xlim(-10, 210)
    ax2.set_xlim(-10, 210)
    ax2.set_ylim(-0.05, 1.05)
    ax2.grid()
    ax2.set_xlabel("pT [MeV]", fontsize=14)
    
    ax1.set_ylabel('Number of Events', fontsize=13)
    
    ax1.set_title(f'{m} MeV TargetSP Recoil $e^{{-}}$ Transverse Momentum' if m!=0 else 'PN Bkg TargetSP Recoil $e^{{-}}$ Transverse Momentum', fontsize=15)
    ax2.set_ylabel('Ratio', fontsize=13)
    plt.tight_layout()
    plt.savefig(f'pT_bias_{m}', facecolor='w', dpi=250)

<IPython.core.display.Javascript object>

/tmp/ipykernel_131667/1905495156.py:30: RuntimeWarning: invalid value encountered in divide
  ratio_arr = val / n1
/tmp/ipykernel_131667/1905495156.py:31: RuntimeWarning: invalid value encountered in divide
  err = (val / n1) * ( (1/np.sqrt(val)) + (1/np.sqrt(n1)) )
/tmp/ipykernel_131667/1905495156.py:31: RuntimeWarning: divide by zero encountered in divide
  err = (val / n1) * ( (1/np.sqrt(val)) + (1/np.sqrt(n1)) )
/tmp/ipykernel_131667/1905495156.py:31: RuntimeWarning: invalid value encountered in multiply
  err = (val / n1) * ( (1/np.sqrt(val)) + (1/np.sqrt(n1)) )


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [25]:
pn_sig_disc = {}

for m in masses[1:]:
    pn_sig_disc[m] = a['ParticleNet_disc'][test_extra_labels==m]

pn_sig_pass = {}
for m in masses[1:]:
    pn_sig_pass[m] = np.sum( (pn_sig_disc[m] > disc_threshold) * (a['TargetSPRecoilE_pt'][test_extra_labels==m] != -999) * (a['maxPE'][test_extra_labels==m] < 8) )

pn_sig_eff = {}
for m in masses[1:]:
    pn_sig_eff[m] = (pn_sig_pass[m] / nEvents[m]) * presel_eff[m]
    
print(pn_sig_eff)

{1: 0.8846609430839114, 10: 0.9004423753350229, 100: 0.8961530509411243, 1000: 0.7745678655886117}
